# 1. Covid-19 Case and Death Rates By State

Coronavirus (COVID-19) is an infectious respiratory disease caused by the SARS-CoV-2 virus. The virus, being highly infectious, sparked a global pandemic and caused mandatory quarantines around the globe. By the end of 2020, vaccines were approved for administration and over time, have proven to decrease the chance of infection and the severity of the symptoms. However, COVID-19 is still an issue to this day and we have seen sporatic spikes in COVID-19 contraction since. COVID-19 has over 6 million world wide confirmed deaths and over 569 million cases of the disease, recorded in July 2022, over the two years the disease has been prevalent. The United States, covered in this project, has over 1 million of those deaths.

This project aims to explore COVID-19 data collected from the CDC and healthdata.gov to analyze previous COVID-19 data, interpret trends between state data, and accurately forecast future cases and deaths in the United States. This notebook, when run, will extract the data straight from the site, updating the model each time it is run. It will also update the Tableau dashboard with its new information, creating an up to date analysis of data each time the notebook it run.


## 1.1 Imports

This notebook aims to import the raw data and clean the messy data in preparation for data analysis and machine learning.

The three datasets are as follows:
1. Covid-19 Reported Patient Impact and Hospital Capacity by State Timeseries collected from healthdata.gov
<br><tab>https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh
2. Vaccine Distribution and Administration by State collected from the CDC
<br><tab>https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc
3. Covid Cases and Deaths Over Time collected from the CDC
<br><tab>https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

We import three datasets into our notebook directly from the site using pandas and its .read_csv() function. **Every time this notebook is run, it will redownload the raw data, including whatever data is new since the last download.**

In [1]:
import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
covid_impact_raw = pd.read_csv('https://healthdata.gov/api/views/g62h-syeh/rows.csv?accessType=DOWNLOAD')
covid_vaccine_raw = pd.read_csv('https://data.cdc.gov/api/views/unsk-b7fc/rows.csv?accessType=DOWNLOAD')
covid_case_deaths_raw = pd.read_csv('https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD')

## 1.2 Data Exploration

The first step before any analysis can be conducted is to explore each imported dataset and clean it for unnecessary unknown or null values. We also filter the datasets by the relevant columns and merge the three sets into one final table.

### 1.2.1 Hospital and Patient Impact Dataset

In [4]:
covid_impact_raw.head()

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_admission_pediatric_covid_confirmed_5_11,previous_day_admission_pediatric_covid_confirmed_5_11_coverage,previous_day_admission_pediatric_covid_confirmed_unknown,previous_day_admission_pediatric_covid_confirmed_unknown_coverage,staffed_icu_pediatric_patients_confirmed_covid,staffed_icu_pediatric_patients_confirmed_covid_coverage,staffed_pediatric_icu_bed_occupancy,staffed_pediatric_icu_bed_occupancy_coverage,total_staffed_pediatric_icu_beds,total_staffed_pediatric_icu_beds_coverage
0,RI,2021/01/13,6,8,1,6,8,1,20.0,14,...,NaN,0,NaN,0,NaN,0,66.0,14,96.0,14
1,RI,2021/01/10,6,8,1,6,8,1,20.0,14,...,NaN,0,NaN,0,NaN,0,62.0,14,96.0,14
2,MA,2021/01/09,12,89,1,14,87,1,61.0,101,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
3,PR,2021/01/08,0,3,64,0,3,64,3.0,66,...,NaN,0,NaN,0,NaN,0,3.0,3,15.0,3
4,SD,2021/01/07,2,60,3,2,60,3,39.0,62,...,NaN,0,NaN,0,NaN,0,11.0,62,83.0,62


In [5]:
covid_impact_raw.shape

(50339, 135)

In [6]:
impact_cols = list(covid_impact_raw.columns)
impact_cols

['state',
 'date',
 'critical_staffing_shortage_today_yes',
 'critical_staffing_shortage_today_no',
 'critical_staffing_shortage_today_not_reported',
 'critical_staffing_shortage_anticipated_within_week_yes',
 'critical_staffing_shortage_anticipated_within_week_no',
 'critical_staffing_shortage_anticipated_within_week_not_reported',
 'hospital_onset_covid',
 'hospital_onset_covid_coverage',
 'inpatient_beds',
 'inpatient_beds_coverage',
 'inpatient_beds_used',
 'inpatient_beds_used_coverage',
 'inpatient_beds_used_covid',
 'inpatient_beds_used_covid_coverage',
 'previous_day_admission_adult_covid_confirmed',
 'previous_day_admission_adult_covid_confirmed_coverage',
 'previous_day_admission_adult_covid_suspected',
 'previous_day_admission_adult_covid_suspected_coverage',
 'previous_day_admission_pediatric_covid_confirmed',
 'previous_day_admission_pediatric_covid_confirmed_coverage',
 'previous_day_admission_pediatric_covid_suspected',
 'previous_day_admission_pediatric_covid_suspected_

In [7]:
covid_impact_raw.isnull().sum().sort_values(ascending=False).head(10)

geocoded_state                                                      50339
previous_day_admission_pediatric_covid_confirmed_12_17              36875
previous_day_admission_pediatric_covid_confirmed_5_11               36865
previous_day_admission_pediatric_covid_confirmed_0_4                36410
previous_day_admission_pediatric_covid_confirmed_unknown            36296
staffed_icu_pediatric_patients_confirmed_covid                      30149
on_hand_supply_therapeutic_c_bamlanivimab_etesevimab_courses        20537
previous_week_therapeutic_c_bamlanivimab_etesevimab_courses_used    20520
on_hand_supply_therapeutic_b_bamlanivimab_courses                   16299
previous_week_therapeutic_b_bamlanivimab_courses_used               16264
dtype: int64

In [8]:
covid_impact_raw['state'].value_counts()

IN    998
HI    998
NC    998
TX    998
MT    998
MN    998
AL    998
NV    980
KS    967
IL    950
MS    949
WV    948
MO    945
OR    944
CA    943
PR    943
LA    943
WA    940
OK    938
GA    938
NE    938
ME    938
KY    938
MD    938
NJ    938
ND    938
WI    938
PA    938
WY    938
MI    938
OH    938
IA    938
SC    937
VA    937
AZ    937
RI    936
AR    933
FL    927
ID    926
NM    925
CO    925
NY    925
TN    925
VT    922
CT    921
UT    919
AK    916
SD    914
NH    912
MA    912
DE    912
DC    911
VI    900
AS    397
Name: state, dtype: int64

The COVID Hospital Impact dataset contains over 46,000 individual records and 135 features.

The data is collected daily since 01-01-2020 with additional columns added over time as discussed on the healthdata.gov page containing the dataset. Certain states seem to have more collected values than others due lack of regular collection and reporting of data.

The potential columns of relevance are 
'state',
 'date', 
 'critical_staffing_shortage_today_yes',
 'critical_staffing_shortage_today_no',
 'critical_staffing_shortage_today_not_reported',
 'critical_staffing_shortage_anticipated_within_week_yes',
 'critical_staffing_shortage_anticipated_within_week_no',
 'inpatient_beds_used_covid',
 'inpatient_beds_used_covid_coverage',
 'previous_day_admission_adult_covid_confirmed',
 'previous_day_admission_adult_covid_confirmed_coverage',
 'previous_day_admission_adult_covid_suspected',
 'previous_day_admission_adult_covid_suspected_coverage',
 'inpatient_bed_covid_utilization'
 'adult_icu_bed_covid_utilization'
 
The column descriptions can be found at the following site:
https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh

In [9]:
cols = ['state', 'date', 'critical_staffing_shortage_today_yes', 'critical_staffing_shortage_today_no', 
        'critical_staffing_shortage_today_not_reported', 'critical_staffing_shortage_anticipated_within_week_yes', 
        'critical_staffing_shortage_anticipated_within_week_no', 'inpatient_beds_used_covid', 
        'inpatient_beds_used_covid_coverage', 'previous_day_admission_adult_covid_confirmed', 
        'previous_day_admission_adult_covid_confirmed_coverage', 'previous_day_admission_adult_covid_suspected', 
        'previous_day_admission_adult_covid_suspected_coverage', 'inpatient_bed_covid_utilization',
        'adult_icu_bed_covid_utilization']
filt_covid_impact = covid_impact_raw[cols]

In [10]:
filt_covid_impact.head()

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,inpatient_beds_used_covid,inpatient_beds_used_covid_coverage,previous_day_admission_adult_covid_confirmed,previous_day_admission_adult_covid_confirmed_coverage,previous_day_admission_adult_covid_suspected,previous_day_admission_adult_covid_suspected_coverage,inpatient_bed_covid_utilization,adult_icu_bed_covid_utilization
0,RI,2021/01/13,6,8,1,6,8,434.0,14,35.0,15,0.0,14,0.169996,0.354430
1,RI,2021/01/10,6,8,1,6,8,449.0,14,33.0,15,0.0,14,0.176424,0.306250
2,MA,2021/01/09,12,89,1,14,87,2075.0,101,269.0,102,142.0,101,0.107235,0.324232
3,PR,2021/01/08,0,3,64,0,3,458.0,66,36.0,67,839.0,66,0.049094,0.120827
4,SD,2021/01/07,2,60,3,2,60,247.0,62,23.0,65,16.0,62,0.089105,0.244813


In [11]:
filt_covid_impact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50339 entries, 0 to 50338
Data columns (total 15 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   state                                                   50339 non-null  object 
 1   date                                                    50339 non-null  object 
 2   critical_staffing_shortage_today_yes                    50339 non-null  int64  
 3   critical_staffing_shortage_today_no                     50339 non-null  int64  
 4   critical_staffing_shortage_today_not_reported           50339 non-null  int64  
 5   critical_staffing_shortage_anticipated_within_week_yes  50339 non-null  int64  
 6   critical_staffing_shortage_anticipated_within_week_no   50339 non-null  int64  
 7   inpatient_beds_used_covid                               50255 non-null  float64
 8   inpatient_beds_used_covid_coverage  

In [12]:
# converting the date column to a datetime dtype

filt_covid_impact['date'] = pd.to_datetime(filt_covid_impact['date'])

In [13]:
filt_covid_impact.isnull().sum()

state                                                        0
date                                                         0
critical_staffing_shortage_today_yes                         0
critical_staffing_shortage_today_no                          0
critical_staffing_shortage_today_not_reported                0
critical_staffing_shortage_anticipated_within_week_yes       0
critical_staffing_shortage_anticipated_within_week_no        0
inpatient_beds_used_covid                                   84
inpatient_beds_used_covid_coverage                           0
previous_day_admission_adult_covid_confirmed              6781
previous_day_admission_adult_covid_confirmed_coverage        0
previous_day_admission_adult_covid_suspected              6932
previous_day_admission_adult_covid_suspected_coverage        0
inpatient_bed_covid_utilization                            270
adult_icu_bed_covid_utilization                           7491
dtype: int64

In [14]:
filt_covid_impact[filt_covid_impact['inpatient_beds_used_covid'].isnull()]['date'].value_counts().sort_index().head()

2020-01-01    2
2020-01-02    2
2020-01-03    2
2020-01-04    3
2020-01-05    3
Name: date, dtype: int64

In [15]:
filt_covid_impact = filt_covid_impact[filt_covid_impact['date']>datetime.datetime(2020,1,20)]

In [16]:
filt_covid_impact.shape

(50179, 15)

In [17]:
filt_covid_impact[filt_covid_impact['adult_icu_bed_covid_utilization'].isnull()]

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,inpatient_beds_used_covid,inpatient_beds_used_covid_coverage,previous_day_admission_adult_covid_confirmed,previous_day_admission_adult_covid_confirmed_coverage,previous_day_admission_adult_covid_suspected,previous_day_admission_adult_covid_suspected_coverage,inpatient_bed_covid_utilization,adult_icu_bed_covid_utilization
57,ND,2020-07-22,0,0,52,0,0,51.0,4,NaN,0,NaN,0,0.062044,NaN
58,VI,2020-07-19,0,0,2,0,0,2.0,2,2.0,2,NaN,0,NaN,NaN
59,ND,2020-07-15,0,0,52,0,0,16.0,4,NaN,0,NaN,0,0.025478,NaN
61,KS,2020-07-10,0,0,130,0,0,217.0,129,NaN,0,NaN,0,0.027030,NaN
62,ND,2020-07-05,0,0,52,0,0,33.0,19,NaN,0,NaN,0,0.024000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21675,IA,2020-05-13,0,1,121,0,1,401.0,82,10.0,40,NaN,0,0.061740,NaN
21678,FL,2020-04-09,0,0,263,0,0,2091.0,261,NaN,0,NaN,0,0.053643,NaN
21679,NE,2020-03-19,0,0,4,0,0,0.0,4,NaN,0,NaN,0,0.000000,NaN
21681,IN,2020-05-30,0,0,135,0,0,919.0,115,NaN,0,NaN,0,0.065173,NaN


We see a small amount of null values in the 'inpatient_beds_used_covid' column that are shown to be values in the early 2020's. 

The United States did not see its first confirmed COVID-19 case until 01-20-2020, so I remove all the columns before that date, which removes about 160 rows.

The 'previous_day_admission_adult_covid_confirmed', 'previous_day_admission_adult_covid_suspected', 'adult_icu_bed_covid_utilization' columns have a large amount of null values that all stem from the early to mid 2020's. I choose not to remove these rows due to the information in the 'inpatient_beds_used_covid' that could be essential in the data training. The null values can be Imputed during modeling, but may be useful in certain machine models, like XGBClassifier.

### 1.2.2 Vaccine Distribution and Administration by State Dataset

In [18]:
covid_vaccine_raw.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Novavax,Distributed_Unk_Manuf,Dist_Per_100K,...,Additional_Doses_Unk_Manuf,Second_Booster,Second_Booster_50Plus,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus,Second_Booster_65Plus_Vax_Pct,Second_Booster_Janssen,Second_Booster_Moderna,Second_Booster_Pfizer,Second_Booster_Unk_Manuf
0,09/21/2022,38,WV,4758445,169900,1901220,2680525,6800.0,0,265516,...,129.0,NaN,111569.0,31.3,79925.0,36.7,71.0,56064.0,63447.0,35.0
1,09/21/2022,38,MP,136930,3600,27620,105510,200.0,0,264084,...,0.0,NaN,1448.0,14.6,564.0,22.8,1.0,258.0,1432.0,0.0
2,09/21/2022,38,MA,19701370,627400,7453880,11612990,7100.0,0,285838,...,49.0,NaN,623671.0,37.9,397234.0,46.8,263.0,341641.0,353562.0,10.0
3,09/21/2022,38,MO,13455705,436700,5078660,7929045,11300.0,0,219240,...,196.0,NaN,355316.0,33.9,254019.0,40.5,197.0,164975.0,213749.0,17.0
4,09/21/2022,38,AS,125010,600,24900,99510,0.0,0,263779,...,3.0,NaN,1947.0,23.3,780.0,32.7,0.0,469.0,1582.0,0.0


In [19]:
covid_vaccine_raw.shape

(36376, 96)

In [20]:
covid_vaccine_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36376 entries, 0 to 36375
Data columns (total 96 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Date                                    36376 non-null  object 
 1   MMWR_week                               36376 non-null  int64  
 2   Location                                36376 non-null  object 
 3   Distributed                             36376 non-null  int64  
 4   Distributed_Janssen                     36376 non-null  int64  
 5   Distributed_Moderna                     36376 non-null  int64  
 6   Distributed_Pfizer                      36376 non-null  int64  
 7   Distributed_Novavax                     576 non-null    float64
 8   Distributed_Unk_Manuf                   36376 non-null  int64  
 9   Dist_Per_100K                           36376 non-null  int64  
 10  Distributed_Per_100k_5Plus              35928 non-null  fl

In [21]:
covid_vaccine_raw.columns

Index(['Date', 'MMWR_week', 'Location', 'Distributed', 'Distributed_Janssen',
       'Distributed_Moderna', 'Distributed_Pfizer', 'Distributed_Novavax',
       'Distributed_Unk_Manuf', 'Dist_Per_100K', 'Distributed_Per_100k_5Plus',
       'Distributed_Per_100k_12Plus', 'Distributed_Per_100k_18Plus',
       'Distributed_Per_100k_65Plus', 'Administered', 'Administered_5Plus',
       'Administered_12Plus', 'Administered_18Plus', 'Administered_65Plus',
       'Administered_Janssen', 'Administered_Moderna', 'Administered_Pfizer',
       'Administered_Novavax', 'Administered_Unk_Manuf', 'Admin_Per_100K',
       'Admin_Per_100k_5Plus', 'Admin_Per_100k_12Plus',
       'Admin_Per_100k_18Plus', 'Admin_Per_100k_65Plus', 'Recip_Administered',
       'Administered_Dose1_Recip', 'Administered_Dose1_Pop_Pct',
       'Administered_Dose1_Recip_5Plus',
       'Administered_Dose1_Recip_5PlusPop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Admini

The COVID Vaccination in the U.S. by State dataset contains over 35,000 individual records and 93 features.

The data is collected daily since 12-13-2020, beginning with weekly data reporting after 06-15-2022.

The potential columns of relevance are 
'Date', 'Location', 'Distributed', 'Dist_Per_100K', 'Administered', 'Administered_Janssen', 
        'Administered_Moderna', 'Administered_Pfizer', 'Admin_Per_100K', 'Recip_Administered', 
        'Series_Complete_Yes', 'Series_Complete_Pop_Pct', 'Additional_Doses', 'Second_Booster'
 
The column descriptions can be found at the following site:
https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

In [22]:
cols = ['Date', 'Location', 'Distributed', 'Dist_Per_100K', 'Administered', 'Administered_Janssen', 
        'Administered_Moderna', 'Administered_Pfizer', 'Admin_Per_100K', 'Recip_Administered', 
        'Series_Complete_Yes', 'Series_Complete_Pop_Pct', 'Additional_Doses']
filt_covid_vaccine = covid_vaccine_raw[cols]

In [23]:
filt_covid_vaccine.isnull().sum()

Date                           0
Location                       0
Distributed                    0
Dist_Per_100K                  0
Administered                   0
Administered_Janssen           0
Administered_Moderna           0
Administered_Pfizer            0
Admin_Per_100K                 0
Recip_Administered             0
Series_Complete_Yes            0
Series_Complete_Pop_Pct        0
Additional_Doses           16348
dtype: int64

In [24]:
filt_covid_vaccine = filt_covid_vaccine.fillna(0)

In [25]:
filt_covid_vaccine.isnull().sum()

Date                       0
Location                   0
Distributed                0
Dist_Per_100K              0
Administered               0
Administered_Janssen       0
Administered_Moderna       0
Administered_Pfizer        0
Admin_Per_100K             0
Recip_Administered         0
Series_Complete_Yes        0
Series_Complete_Pop_Pct    0
Additional_Doses           0
dtype: int64

The large null values for additional_doses and second_booster can be attributed to the lack of additional doses and second boosters administered at the beginning of the pandemic. These null values can be imputed to 0's.

In [26]:
# The column names are readjusted for consistent naming

col_dict = {'Date':'date', 'Location':'state', 'Distributed':'distributed', 'Dist_Per_100K':'dist_per_100k', 
            'Administered':'admin', 'Administered_Janssen':'administered_j', 'Administered_Moderna':'administered_m',
            'Administered_Pfizer':'administered_p', 'Admin_Per_100K':'admin_per_100k', 'Recip_Administered': 'total_admin',
            'Series_Complete_Yes':'fully_vacc', 'Series_Complete_Pop_Pct':'fully_vacc_pop_perc', 
            'Additional_Doses':'first_booster'}
filt_covid_vaccine.rename(columns = col_dict, inplace = True)

In [27]:
filt_covid_vaccine.head()

,date,state,distributed,dist_per_100k,admin,administered_j,administered_m,administered_p,admin_per_100k,total_admin,fully_vacc,fully_vacc_pop_perc,first_booster
0,09/21/2022,WV,4758445,265516,2885733,68269,1242958,1572263,161021,2892428,1053708,58.8,504983.0
1,09/21/2022,MP,136930,264084,112886,1396,15531,95949,217712,113073,43629,84.1,22577.0
2,09/21/2022,MA,19701370,285838,16005329,407881,6293995,9302471,232214,16088523,5599181,81.2,3015796.0
3,09/21/2022,MO,13455705,219240,9537187,253085,3449525,5831615,155394,9520255,3528451,57.5,1586536.0
4,09/21/2022,AS,125010,263779,114136,580,25314,86976,240834,114622,42333,89.3,24469.0


In [28]:
# converting the date column to a datetime dtype

filt_covid_vaccine['date'] = pd.to_datetime(filt_covid_vaccine['date'])

In [29]:
filt_covid_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36376 entries, 0 to 36375
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 36376 non-null  datetime64[ns]
 1   state                36376 non-null  object        
 2   distributed          36376 non-null  int64         
 3   dist_per_100k        36376 non-null  int64         
 4   admin                36376 non-null  int64         
 5   administered_j       36376 non-null  int64         
 6   administered_m       36376 non-null  int64         
 7   administered_p       36376 non-null  int64         
 8   admin_per_100k       36376 non-null  int64         
 9   total_admin          36376 non-null  int64         
 10  fully_vacc           36376 non-null  int64         
 11  fully_vacc_pop_perc  36376 non-null  float64       
 12  first_booster        36376 non-null  float64       
dtypes: datetime64[ns](1), float64(2

### 1.2.3 Covid Cases and Deaths Over Time by State

In [30]:
covid_case_deaths_raw.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,04/25/2020,FL,30453,NaN,NaN,798,0.0,1221,NaN,NaN,39,0.0,09/24/2022 12:34:02 PM,Not agree,Not agree
1,01/14/2022,KS,621273,470516.0,150757.0,19414,6964.0,7162,NaN,NaN,21,4.0,01/15/2022 02:59:30 PM,Agree,NaN
2,02/12/2021,UT,359641,359641.0,0.0,1060,0.0,1785,1729.0,56.0,11,2.0,02/13/2021 02:50:08 PM,Agree,Agree
3,07/26/2021,FL,2541125,NaN,NaN,16373,3700.0,39446,NaN,NaN,111,4.0,07/26/2021 12:00:00 AM,Not agree,Not agree
4,09/01/2021,ND,118491,107475.0,11016.0,536,66.0,1562,NaN,NaN,1,0.0,09/02/2021 01:49:05 PM,Agree,Not agree


In [31]:
covid_case_deaths_raw.shape

(58560, 15)

In [32]:
covid_case_deaths_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58560 entries, 0 to 58559
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   submission_date  58560 non-null  object 
 1   state            58560 non-null  object 
 2   tot_cases        58560 non-null  int64  
 3   conf_cases       33109 non-null  float64
 4   prob_cases       33037 non-null  float64
 5   new_case         58560 non-null  int64  
 6   pnew_case        55034 non-null  float64
 7   tot_death        58560 non-null  int64  
 8   conf_death       32398 non-null  float64
 9   prob_death       32398 non-null  float64
 10  new_death        58560 non-null  int64  
 11  pnew_death       55066 non-null  float64
 12  created_at       58560 non-null  object 
 13  consent_cases    48795 non-null  object 
 14  consent_deaths   49776 non-null  object 
dtypes: float64(6), int64(4), object(5)
memory usage: 6.7+ MB


The COVID Case and Death in the U.S. by State dataset contains over 54,000 individual records and 15 features.

The data is collected daily since 01-23-2020.

The potential columns of relevance are 
'submission_date','state','tot_cases','new_case','pnew_case','tot_death','new_death','pnew_death'

We are choosing not to use the conf_cases, prob_cases, conf_death, and prob_death categories due to some states not consenting to reporting this data (consent marked in the consent_cases and consent_deaths columns)
 
The column descriptions can be found at the following site:
https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

In [33]:
cols = ['submission_date','state','tot_cases','new_case','pnew_case','tot_death','new_death','pnew_death']
filt_covid_case_deaths = covid_case_deaths_raw[cols]

In [34]:
# The column names are readjusted for consistent naming

col_dict = {'submission_date':'date', 'pnew_case':'prob_new_case', 'pnew_death':'prob_new_death'}
filt_covid_case_deaths = filt_covid_case_deaths.rename(columns = col_dict)

In [35]:
filt_covid_case_deaths.head()

,date,state,tot_cases,new_case,prob_new_case,tot_death,new_death,prob_new_death
0,04/25/2020,FL,30453,798,0.0,1221,39,0.0
1,01/14/2022,KS,621273,19414,6964.0,7162,21,4.0
2,02/12/2021,UT,359641,1060,0.0,1785,11,2.0
3,07/26/2021,FL,2541125,16373,3700.0,39446,111,4.0
4,09/01/2021,ND,118491,536,66.0,1562,1,0.0


In [36]:
# converting the date column to a datetime dtype

filt_covid_case_deaths['date'] = pd.to_datetime(filt_covid_case_deaths['date'])

In [37]:
filt_covid_case_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58560 entries, 0 to 58559
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            58560 non-null  datetime64[ns]
 1   state           58560 non-null  object        
 2   tot_cases       58560 non-null  int64         
 3   new_case        58560 non-null  int64         
 4   prob_new_case   55034 non-null  float64       
 5   tot_death       58560 non-null  int64         
 6   new_death       58560 non-null  int64         
 7   prob_new_death  55066 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 3.6+ MB


In [38]:
filt_covid_case_deaths.isnull().sum()

date                 0
state                0
tot_cases            0
new_case             0
prob_new_case     3526
tot_death            0
new_death            0
prob_new_death    3494
dtype: int64

I choose to leave the null values as null for now. Upon creating the machine learning Pipeline, the mean can be imputed for the null values.

## 1.3. Data Merge

Now that we have three cleaned datasets, we merge the sets based off the date and state columns

In [39]:
merge_covid = pd.merge(filt_covid_impact, filt_covid_vaccine, how="left", on=['date', 'state'], suffixes=("_x", "_y"))

In [40]:
merge_covid = pd.merge(merge_covid, filt_covid_case_deaths, how='left', on=['date', 'state'], suffixes=("_x", "_y"))

In [41]:
merge_covid.isnull().sum()

state                                                         0
date                                                          0
critical_staffing_shortage_today_yes                          0
critical_staffing_shortage_today_no                           0
critical_staffing_shortage_today_not_reported                 0
critical_staffing_shortage_anticipated_within_week_yes        0
critical_staffing_shortage_anticipated_within_week_no         0
inpatient_beds_used_covid                                    54
inpatient_beds_used_covid_coverage                            0
previous_day_admission_adult_covid_confirmed               6631
previous_day_admission_adult_covid_confirmed_coverage         0
previous_day_admission_adult_covid_suspected               6782
previous_day_admission_adult_covid_suspected_coverage         0
inpatient_bed_covid_utilization                             218
adult_icu_bed_covid_utilization                            7341
distributed                             

The new null values due to merging are due to the vaccination schedule changing from reporting daily to weekly.

## 1.4. Cleaned Merged DataFrame Save

In [42]:
merge_covid = merge_covid.reset_index(drop=True)
merge_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50179 entries, 0 to 50178
Data columns (total 32 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   state                                                   50179 non-null  object        
 1   date                                                    50179 non-null  datetime64[ns]
 2   critical_staffing_shortage_today_yes                    50179 non-null  int64         
 3   critical_staffing_shortage_today_no                     50179 non-null  int64         
 4   critical_staffing_shortage_today_not_reported           50179 non-null  int64         
 5   critical_staffing_shortage_anticipated_within_week_yes  50179 non-null  int64         
 6   critical_staffing_shortage_anticipated_within_week_no   50179 non-null  int64         
 7   inpatient_beds_used_covid                               50

In [43]:
merge_covid.to_csv('../clean_data/merge_covid.csv')

## 1.5. Continuation

Now that the datasets are merged and cleaned, we export the CSV and continue the analysis in the next notebook where I will focus on Exploratory Data Analysis and 